In [1]:
import pathlib
pathlib.Path().absolute()

PosixPath('/Users/amberteetsel/Columbia_Data/world-weather-analysis/Challenge/Vacation_Search')

In [4]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config2 import gkey

# Configure gmaps API key
gmaps.configure(api_key=gkey)

In [5]:
# 1. Import the WeatherPy_database.csv file. 
city_df = pd.read_csv("~/Columbia_Data/world-weather-analysis/Challenge/Weather_Database/WeatherPy_Database.csv")
city_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Spearfish,US,44.4908,-103.8594,37.99,25,1,12.66,clear sky
1,1,Neyvo-Shaytanskiy,RU,57.7286,61.2496,9.07,95,100,2.33,overcast clouds
2,2,Port Alfred,ZA,-33.5906,26.8910,68.00,89,56,18.16,broken clouds
3,3,Mombetsu,JP,44.3525,143.3525,27.00,75,100,3.00,overcast clouds
4,4,Fortuna,US,40.5982,-124.1573,57.20,88,1,6.58,clear sky


In [7]:
# 2. Prompt the user to enter minimum and maximum temperature criteria
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 80


In [8]:
# 3. Filter the city_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_temp_df = city_df.loc[(city_df["Max Temp"] >= min_temp) & \
                                (city_df["Max Temp"] <= max_temp)]
preferred_temp_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
7,7,Georgetown,MY,5.4112,100.3354,78.01,88,20,8.05,few clouds
13,13,Mataura,NZ,-46.1927,168.8643,70.30,50,16,2.17,few clouds
15,15,Rikitea,PF,-23.1203,-134.9692,78.91,66,21,7.27,few clouds
23,23,Kapaa,US,22.0752,-159.3190,79.00,78,1,6.91,clear sky
27,27,Paciran,ID,-6.8767,112.3761,77.29,85,100,6.78,overcast clouds


In [9]:
# 4a. Determine if there are any empty rows.
preferred_temp_df.isnull().sum()

City_ID                0
City                   0
Country                1
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [10]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_temp_df.dropna()

In [11]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
7,Georgetown,MY,78.01,few clouds,5.4112,100.3354,
13,Mataura,NZ,70.30,few clouds,-46.1927,168.8643,
15,Rikitea,PF,78.91,few clouds,-23.1203,-134.9692,
23,Kapaa,US,79.00,clear sky,22.0752,-159.3190,
27,Paciran,ID,77.29,overcast clouds,-6.8767,112.3761,
32,Vao,NC,78.12,overcast clouds,-22.6667,167.4833,
33,Carnarvon,AU,78.80,clear sky,-24.8667,113.6333,
35,Singaraja,ID,79.95,overcast clouds,-8.1120,115.0882,
40,Kavieng,PG,76.26,heavy intensity rain,-2.5744,150.7967,
45,Saint-Philippe,RE,78.80,clear sky,-21.3585,55.7679,


#### Note: The code below runs extremely slowly on my machine and makes it difficult to complete the assignment. I discussed the issue with a TA and we were't able to resolve it.

In [14]:
# 6a. Set parameters to search for hotels within 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": gkey
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row['Lat']
    lng = row['Lng']
    params['location'] = f"{lat}, {lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print(f"Hotel not found... Skipping")

Hotel not found... Skipping
Hotel not found... Skipping
Hotel not found... Skipping
Hotel not found... Skipping
Hotel not found... Skipping


KeyboardInterrupt: 

In [15]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df[hotel_df['Hotel Name'].notna()]
clean_hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
7,Georgetown,MY,78.01,few clouds,5.4112,100.3354,Cititel Penang
13,Mataura,NZ,70.30,few clouds,-46.1927,168.8643,Ellie's Villa
15,Rikitea,PF,78.91,few clouds,-23.1203,-134.9692,Pension Maro'i
23,Kapaa,US,79.00,clear sky,22.0752,-159.3190,Sheraton Kauai Resort at Coconut Beach
27,Paciran,ID,77.29,overcast clouds,-6.8767,112.3761,Tanjung Kodok Beach Resort


In [19]:
# 8a. Create the output File (CSV)
output_data_file = "~/Columbia_Data/world-weather-analysis/Challenge/Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [27]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [29]:
# 11a. Add a marker layer for each city to the map.
locations = clean_hotel_df[['Lat', 'Lng']]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))

#### Please note that not every marker will have a hotel name due to issues with the runtime of the 